In [1]:
import time
import random
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import requests
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import clear_output
import openai

<h1 style="font-size:40px;"> <center> CONSTANTS </center> </h1>

In [54]:
KEYWORD = "cinayet"
START_DATE = "01/01/2016"
FINISH_DATE = "31/12/2016"
MAIN_CATEGORY = "gundem"    
file_name = KEYWORD + "-" + START_DATE[-4:] + "-" + MAIN_CATEGORY

<h1 style="font-size:40px;"> <center> USER AGENTS </center> </h1>

In [5]:
user_agents = [
"Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/525.19 (KHTML, like Gecko) Chrome/1.0.154.53 Safari/525.19",
"Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/525.19 (KHTML, like Gecko) Chrome/1.0.154.36 Safari/525.19",
"Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.10 (KHTML, like Gecko) Chrome/7.0.540.0 Safari/534.10",
"Mozilla/5.0 (Windows; U; Windows NT 5.2; en-US) AppleWebKit/534.4 (KHTML, like Gecko) Chrome/6.0.481.0 Safari/534.4",
"Mozilla/5.0 (Macintosh; U; Intel Mac OS X; en-US) AppleWebKit/533.4 (KHTML, like Gecko) Chrome/5.0.375.86 Safari/533.4",
"Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/532.2 (KHTML, like Gecko) Chrome/4.0.223.3 Safari/532.2",
"Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/4.0.201.1 Safari/532.0",
"Mozilla/5.0 (Windows; U; Windows NT 5.2; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/3.0.195.27 Safari/532.0",
"Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/530.5 (KHTML, like Gecko) Chrome/2.0.173.1 Safari/530.5",
"Mozilla/5.0 (Windows; U; Windows NT 5.2; en-US) AppleWebKit/534.10 (KHTML, like Gecko) Chrome/8.0.558.0 Safari/534.10",
"Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/540.0 (KHTML,like Gecko) Chrome/9.1.0.0 Safari/540.0",
"Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/534.14 (KHTML, like Gecko) Chrome/9.0.600.0 Safari/534.14",
"Mozilla/5.0 (X11; U; Windows NT 6; en-US) AppleWebKit/534.12 (KHTML, like Gecko) Chrome/9.0.587.0 Safari/534.12",
"Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.13 (KHTML, like Gecko) Chrome/9.0.597.0 Safari/534.13",
"Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.11 Safari/534.16",
"Mozilla/5.0 (Windows; U; Windows NT 6.0; en-US) AppleWebKit/534.20 (KHTML, like Gecko) Chrome/11.0.672.2 Safari/534.20",
"Mozilla/5.0 (Windows NT 6.0) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/14.0.792.0 Safari/535.1",
"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/535.2 (KHTML, like Gecko) Chrome/15.0.872.0 Safari/535.2",
"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.7 (KHTML, like Gecko) Chrome/16.0.912.36 Safari/535.7",
"Mozilla/5.0 (Windows NT 6.0; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.66 Safari/535.11",
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8) AppleWebKit/535.19 (KHTML, like Gecko) Chrome/18.0.1025.45 Safari/535.19",
"Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
"Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
"Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.15 (KHTML, like Gecko) Chrome/24.0.1295.0 Safari/537.15",
"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36",
"Mozilla/5.0 (Windows NT 6.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1467.0 Safari/537.36",
"Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/30.0.1599.101 Safari/537.36",
"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/31.0.1623.0 Safari/537.36",
"Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.116 Safari/537.36",
"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36",
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/40.0.2214.38 Safari/537.36",
"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.71 Safari/537.36",
"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36",
"Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.62 Safari/537.36",
"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36",
"Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10.5; en-US; rv:1.9.1b3) Gecko/20090305 Firefox/3.1b3 GTB5",
"Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10.5; ko; rv:1.9.1b2) Gecko/20081201 Firefox/3.1b2",
"Mozilla/5.0 (X11; U; SunOS sun4u; en-US; rv:1.9b5) Gecko/2008032620 Firefox/3.0b5",
"Mozilla/5.0 (X11; U; Linux x86_64; en-US; rv:1.8.1.12) Gecko/20080214 Firefox/2.0.0.12",
"Mozilla/5.0 (Windows; U; Windows NT 5.1; cs; rv:1.9.0.8) Gecko/2009032609 Firefox/3.0.8",
"Mozilla/5.0 (X11; U; OpenBSD i386; en-US; rv:1.8.0.5) Gecko/20060819 Firefox/1.5.0.5",
"Mozilla/5.0 (Windows; U; Windows NT 5.0; es-ES; rv:1.8.0.3) Gecko/20060426 Firefox/1.5.0.3",
"Mozilla/5.0 (Windows; U; WinNT4.0; en-US; rv:1.7.9) Gecko/20050711 Firefox/1.0.5",
"Mozilla/5.0 (Windows; Windows NT 6.1; rv:2.0b2) Gecko/20100720 Firefox/4.0b2",
"Mozilla/5.0 (X11; Linux x86_64; rv:2.0b4) Gecko/20100818 Firefox/4.0b4",
"Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.2) Gecko/20100308 Ubuntu/10.04 (lucid) Firefox/3.6 GTB7.1",
"Mozilla/5.0 (Windows NT 6.1; WOW64; rv:2.0b7) Gecko/20101111 Firefox/4.0b7",
"Mozilla/5.0 (Windows NT 6.1; WOW64; rv:2.0b8pre) Gecko/20101114 Firefox/4.0b8pre",
"Mozilla/5.0 (X11; Linux x86_64; rv:2.0b9pre) Gecko/20110111 Firefox/4.0b9pre",
"Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:2.0b9pre) Gecko/20101228 Firefox/4.0b9pre",
"Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:2.2a1pre) Gecko/20110324 Firefox/4.2a1pre",
"Mozilla/5.0 (X11; U; Linux amd64; rv:5.0) Gecko/20100101 Firefox/5.0 (Debian)",
"Mozilla/5.0 (Windows NT 6.1; WOW64; rv:6.0a2) Gecko/20110613 Firefox/6.0a2",
"Mozilla/5.0 (X11; Linux i686 on x86_64; rv:12.0) Gecko/20100101 Firefox/12.0",
"Mozilla/5.0 (Windows NT 6.1; rv:15.0) Gecko/20120716 Firefox/15.0a2",
"Mozilla/5.0 (X11; Ubuntu; Linux armv7l; rv:17.0) Gecko/20100101 Firefox/17.0",
"Mozilla/5.0 (Windows NT 6.1; rv:21.0) Gecko/20130328 Firefox/21.0",
"Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:22.0) Gecko/20130328 Firefox/22.0",
"Mozilla/5.0 (Windows NT 5.1; rv:25.0) Gecko/20100101 Firefox/25.0",
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10.8; rv:25.0) Gecko/20100101 Firefox/25.0",
"Mozilla/5.0 (Windows NT 6.1; rv:28.0) Gecko/20100101 Firefox/28.0",
"Mozilla/5.0 (X11; Linux i686; rv:30.0) Gecko/20100101 Firefox/30.0",
"Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20100101 Firefox/31.0",
"Mozilla/5.0 (Windows NT 6.1; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0",
"Mozilla/5.0 (Windows NT 10.0; WOW64; rv:40.0) Gecko/20100101 Firefox/40.0",
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:58.0) Gecko/20100101 Firefox/58.0"
]

<h1 style="font-size:40px;"> <center> COLLECT NEWS URLS </center> </h1>

In [48]:
start = time.time()

urls = []
titles = []
abstracts = []
dates = []

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
service = Service('/chromedriver.exe')
service.start()
driver = webdriver.Remote(service.service_url, chrome_options=options)
pages = list(range(1,180))
last_url_count = 0

for page in pages:
    try:
        search_url = f"https://www.hurriyet.com.tr/arama/#/?page={page}&key={KEYWORD}&order=Yeniden%20Eskiyewhere=hurriyet&how=Article&and={KEYWORD}&startDate={START_DATE}&finishDate={FINISH_DATE}&platform=hurriyet&mainCategory=/{MAIN_CATEGORY}/&isDetail=true"
        driver.get(search_url)
        time.sleep(3)
        div_content_tags = driver.find_elements(By.XPATH,"//div[@class='hs-cnn-content']")
        for div_content_tag in div_content_tags:
            a_tag = div_content_tag.find_element(By.TAG_NAME,"a")
            urls.append(a_tag.get_attribute("href"))
        p_title_tags = driver.find_elements(By.XPATH,"//p[@class='hs-cnnc-title']")
        for p_title_tag in p_title_tags:
            titles.append(p_title_tag.get_attribute("innerHTML"))
        p_text_tags = driver.find_elements(By.XPATH,"//p[@class='hs-cnnc-text']")
        for p_text_tag in p_text_tags:
            abstracts.append(p_text_tag.get_attribute("innerHTML"))
        p_date_tags = driver.find_elements(By.XPATH,"//p[@class='hs-cnncc-date']")
        for p_date_tag in p_date_tags:
            dates.append(p_date_tag.get_attribute("innerHTML")) 
        if last_url_count == len(urls):
            break
        last_url_count = len(urls)
        clear_output(wait=True)
        print(f"{len(urls)} urls were fetched so far (page number-{page})", sep=' ', end='', flush=True)
    except Exception as e: 
        print(e)
        print("Last Page")
        break
    
driver.quit()  
print(f"\n\n{len(urls)} urls were collected in {time.time() - start} second.")
print(f"{round(len(urls) / (time.time() - start), 2)} urls were collected per second on average.")

1592 urls were fetched so far (page number-160)

1592 urls were collected in 650.4402616024017 second.
2.45 urls were collected per second on average.


In [49]:
scraped_urls = pd.DataFrame({"url":urls, "title":titles, "abstract":abstracts, "date":dates })
#scraped_urls.set_index("date")
scraped_urls

,url,title,abstract,date
0,http://www.hurriyet.com.tr/gundem/ankarada-sil...,"Ankara’da silahlı kavga: 3 ölü, 1 yaralı",Ankara’nın Mamak ilçesinde bir kafede çıkan si...,31.12.2022
1,http://www.hurriyet.com.tr/gundem/copu-dokmeye...,Çöpü dökmeyen kardeşini 38 yerinden bıçaklayar...,Ankara'da çöpü dökmesini isteyince kendisini t...,30.12.2022
2,http://www.hurriyet.com.tr/gundem/bayrampasada...,Bayrampaşa'da korkunç olay! Önce darbettiler s...,İstanbul Bayrampaşa'da bir eğlence mekanında M...,30.12.2022
3,http://www.hurriyet.com.tr/gundem/silanin-kati...,Sıla’nın katiline ağırlaştırılmış müebbet,16 yaşındaki Sıla Şentürk’ü boğazını keserek ö...,30.12.2022
4,http://www.hurriyet.com.tr/gundem/cem-garipogl...,Cem Garipoğlu'nun mezarının açılmasına verilen...,13 yıl önce vahşice öldürülen Münevver Karabul...,29.12.2022
...,...,...,...,...
1587,http://www.hurriyet.com.tr/gundem/kirsehirde-a...,"Kırşehir'de ağabey-kardeş, 23 gün arayla öldür...",Kırşehir'in Kaman ilçesinde husumetlileri tara...,2.1.2022
1588,http://www.hurriyet.com.tr/gundem/eglence-cina...,Eğlence cinayetine ömür boyu hapis cezası,"Bursa’da, arkadaşlarıyla eğlenmek için gittiği...",1.1.2022
1589,http://www.hurriyet.com.tr/gundem/bagcilarda-y...,"Bağcılar’daki cinayet, intihar çıktı: Hepinizi...",Bağcılar’da yılbaşı gecesi 14 yaşındaki Hüseyi...,1.1.2022
1590,http://www.hurriyet.com.tr/gundem/arkadas-grub...,Arkadaş grubunun yılbaşı eğlencesinde bıçaklı ...,"Tokat'ın Erbaa ilçesinde Nurettin Öztürk (56),...",1.1.2022


In [50]:
scraped_urls.to_excel(f"outputs/without detailed content/xlsx/scraped_urls-{file_name}.xlsx")
scraped_urls.to_csv(f"outputs/without detailed content/csv/scraped_urls-{file_name}.csv")

<h1 style="font-size:40px;"> <center> COLLECT NEWS CONTENTS </center> </h1>

In [38]:
scraped_urls = pd.read_csv(f"outputs/without detailed content/csv/scraped_urls-{file_name}.csv").drop(["Unnamed: 0"], axis=1)
scraped_urls

,url,title,abstract,date
0,http://www.hurriyet.com.tr/gundem/kiz-bulacagi...,'Kız bulacağım' dedi öldürüp çöpe attı,"Bursa’da 9 aydır aranan Hüseyin Altı’yı, arkad...",30.12.2016
1,http://www.hurriyet.com.tr/gundem/kaderin-kati...,Kader'in katil zanlısının savunması çileden çı...,Eskişehir'de ormanlık alana götürdüğü 17 yaşın...,30.12.2016
2,http://www.hurriyet.com.tr/gundem/bir-annenin-...,Bir annenin en acı isteği... 'Bir kurşun da ba...,Eskişehir'de 17 yaşındaki Kader Kaya'ya tecavü...,30.12.2016
3,http://www.hurriyet.com.tr/gundem/istanbuldaki...,İstanbul’daki vahşi cinayet 3 yıl sonra ortaya...,İstanbul’da 3 yıl önce esrarengiz bir şekilde ...,30.12.2016
4,http://www.hurriyet.com.tr/gundem/korkunc-cina...,Korkunç cinayet aylar sonra çözüldü! Katilden ...,"Bursa'da gözaltına alınan zanlı, yaklaşık 9 ay...",30.12.2016
...,...,...,...,...
1277,http://www.hurriyet.com.tr/gundem/hrant-dink-d...,Hrant Dink davasında çelişkili ifadeler,"MİT’çi: Genelkurmay evrak almaya gönderdi, tes...",3.1.2016
1278,http://www.hurriyet.com.tr/gundem/istanbulda-s...,İstanbul'da Suudi Arabistan protestosu,"Dünya Caferi Alimler Birliği (CABİR) üyeleri, ...",3.1.2016
1279,http://www.hurriyet.com.tr/gundem/is-adami-vur...,"İş adamı vuruldu, kurt köpekli komşu kaçtı",Beykoz'da&nbsp;Baklacı Mahallesi Evren Sokak’t...,2.1.2016
1280,http://www.hurriyet.com.tr/gundem/kar-temizley...,Kar temizleyen işçiler son yolculuklarına uğur...,TEM Otoyolu Ümraniye mevkiinde kar temizleme ç...,2.1.2016


In [39]:
start = time.time()

long_contents = []
df_dict = scraped_urls.to_dict("records")
for row in df_dict:
    try:
        headers = {
        "User-Agent": random.choice(user_agents),
        }
        url = row["url"]
        response = requests.get(url.strip(), headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        div_tag = soup.find("div", {"class":"news-content readingTime"})
        if div_tag is None:
            div_tag = soup.find("div", {"class":"news-detail-text"})        
        p_tags = div_tag.findAll("p")
        result = ""
        for p_tag in p_tags:
            result += p_tag.text + "\n"
    except:
        result = ""
    long_contents.append(result)
    clear_output(wait=True)
    print(f"Content of {len(long_contents)} urls were fetched so far", sep=' ', end='', flush=True)
    
print(f"\n\nContent of {len(long_contents)} urls were collected in {time.time() - start} second.")
print(f"Content of {round(len(long_contents) / (time.time() - start), 2)} urls were collected per second on average.")

Content of 1282 urls were fetched so far

Content of 1282 urls were collected in 721.976884841919 second.
Content of 1.78 urls were collected per second on average.


In [40]:
scraped_urls["long-content"] = long_contents
scraped_urls

,url,title,abstract,date,long-content
0,http://www.hurriyet.com.tr/gundem/kiz-bulacagi...,'Kız bulacağım' dedi öldürüp çöpe attı,"Bursa’da 9 aydır aranan Hüseyin Altı’yı, arkad...",30.12.2016,BURSA’nın Nilüfer ilçesinde yaşayan inşaat işç...
1,http://www.hurriyet.com.tr/gundem/kaderin-kati...,Kader'in katil zanlısının savunması çileden çı...,Eskişehir'de ormanlık alana götürdüğü 17 yaşın...,30.12.2016,Olay geçen 28 Temmuz’da Eskişehir’in kent merk...
2,http://www.hurriyet.com.tr/gundem/bir-annenin-...,Bir annenin en acı isteği... 'Bir kurşun da ba...,Eskişehir'de 17 yaşındaki Kader Kaya'ya tecavü...,30.12.2016,Eskişehir’de amatör olarak fotomodellik yapan ...
3,http://www.hurriyet.com.tr/gundem/istanbuldaki...,İstanbul’daki vahşi cinayet 3 yıl sonra ortaya...,İstanbul’da 3 yıl önce esrarengiz bir şekilde ...,30.12.2016,Olaya karışan 12 kişi polis tarafından gözaltı...
4,http://www.hurriyet.com.tr/gundem/korkunc-cina...,Korkunç cinayet aylar sonra çözüldü! Katilden ...,"Bursa'da gözaltına alınan zanlı, yaklaşık 9 ay...",30.12.2016,Nilüfer ilçesi Çalı Mahallesi'nde yaşayan inşa...
...,...,...,...,...,...
1277,http://www.hurriyet.com.tr/gundem/hrant-dink-d...,Hrant Dink davasında çelişkili ifadeler,"MİT’çi: Genelkurmay evrak almaya gönderdi, tes...",3.1.2016,HRANT Dink’in 2004 yılında İstanbul Valiliği’n...
1278,http://www.hurriyet.com.tr/gundem/istanbulda-s...,İstanbul'da Suudi Arabistan protestosu,"Dünya Caferi Alimler Birliği (CABİR) üyeleri, ...",3.1.2016,Suudi Arabistan'ın İstanbul Başkonsolosluğu ön...
1279,http://www.hurriyet.com.tr/gundem/is-adami-vur...,"İş adamı vuruldu, kurt köpekli komşu kaçtı",Beykoz'da&nbsp;Baklacı Mahallesi Evren Sokak’t...,2.1.2016,\n3 gün önce Romanya’dan gelen babaları Vural’...
1280,http://www.hurriyet.com.tr/gundem/kar-temizley...,Kar temizleyen işçiler son yolculuklarına uğur...,TEM Otoyolu Ümraniye mevkiinde kar temizleme ç...,2.1.2016,Abdullah Ceylan (52) ve Alican Gürkan'ın (53) ...


In [42]:
scraped_urls.to_excel(f"outputs/with detailed content/xlsx/scraped_urls_with_detailed_content-{file_name}.xlsx", engine='xlsxwriter')
scraped_urls.to_csv(f"outputs/with detailed content/csv/scraped_urls_with_detailed_content-{file_name}.csv")

<h1 style="font-size:40px;"> <center> TEXT ANALYZING </center> </h1>

In [55]:
df = pd.read_csv(f"outputs/with detailed content/csv/scraped_urls_with_detailed_content-{file_name}.csv").drop("Unnamed: 0", axis=1)
df

,url,title,abstract,date,long-content
0,http://www.hurriyet.com.tr/gundem/kiz-bulacagi...,'Kız bulacağım' dedi öldürüp çöpe attı,"Bursa’da 9 aydır aranan Hüseyin Altı’yı, arkad...",30.12.2016,BURSA’nın Nilüfer ilçesinde yaşayan inşaat işç...
1,http://www.hurriyet.com.tr/gundem/kaderin-kati...,Kader'in katil zanlısının savunması çileden çı...,Eskişehir'de ormanlık alana götürdüğü 17 yaşın...,30.12.2016,Olay geçen 28 Temmuz’da Eskişehir’in kent merk...
2,http://www.hurriyet.com.tr/gundem/bir-annenin-...,Bir annenin en acı isteği... 'Bir kurşun da ba...,Eskişehir'de 17 yaşındaki Kader Kaya'ya tecavü...,30.12.2016,Eskişehir’de amatör olarak fotomodellik yapan ...
3,http://www.hurriyet.com.tr/gundem/istanbuldaki...,İstanbul’daki vahşi cinayet 3 yıl sonra ortaya...,İstanbul’da 3 yıl önce esrarengiz bir şekilde ...,30.12.2016,Olaya karışan 12 kişi polis tarafından gözaltı...
4,http://www.hurriyet.com.tr/gundem/korkunc-cina...,Korkunç cinayet aylar sonra çözüldü! Katilden ...,"Bursa'da gözaltına alınan zanlı, yaklaşık 9 ay...",30.12.2016,Nilüfer ilçesi Çalı Mahallesi'nde yaşayan inşa...
...,...,...,...,...,...
1277,http://www.hurriyet.com.tr/gundem/hrant-dink-d...,Hrant Dink davasında çelişkili ifadeler,"MİT’çi: Genelkurmay evrak almaya gönderdi, tes...",3.1.2016,HRANT Dink’in 2004 yılında İstanbul Valiliği’n...
1278,http://www.hurriyet.com.tr/gundem/istanbulda-s...,İstanbul'da Suudi Arabistan protestosu,"Dünya Caferi Alimler Birliği (CABİR) üyeleri, ...",3.1.2016,Suudi Arabistan'ın İstanbul Başkonsolosluğu ön...
1279,http://www.hurriyet.com.tr/gundem/is-adami-vur...,"İş adamı vuruldu, kurt köpekli komşu kaçtı",Beykoz'da&nbsp;Baklacı Mahallesi Evren Sokak’t...,2.1.2016,\n3 gün önce Romanya’dan gelen babaları Vural’...
1280,http://www.hurriyet.com.tr/gundem/kar-temizley...,Kar temizleyen işçiler son yolculuklarına uğur...,TEM Otoyolu Ümraniye mevkiinde kar temizleme ç...,2.1.2016,Abdullah Ceylan (52) ve Alican Gürkan'ın (53) ...


In [56]:
def generate_response(prompt: str = "example prompt"):
    """
    Args:
        prompt (str): user input prompt

    Returns:
        str: response of chatgpt
    """
    completions = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=50,
        n=1,
        stop=None,
        temperature=0,
    )
    return completions.choices[0].text

In [57]:
with open('key.txt') as f:
    openai.api_key = f.readlines()[0]
answers = []

In [78]:
start = time.time()


for i in range(len(df)):
    try:
        question = """Please tell me the name and gender of the person who kills, 
        the name and the gender of the person who is killed, and date (DD.MM.YYYY) when the murder happened respectively  
        in a uniform format like this: name_of_killer, gender_of killer, name_of_killed, gender_of_killed, date 
        (dont break the order) according to this news. : (Thank you) """
        question += df.iloc[i]["long-content"]
        answer = generate_response(question)
        result = str(i) + " - " + answer + "\n"
        time.sleep(2)
        print(result)
        answers.append(answer)
    except:
        answers.append("long text")
    
print(f"\n\nContent of {i} urls were analyzed in {time.time() - start} second.")
print(f"Content of {round(i / (time.time() - start), 2)} urls were analyzed per second on average.")

258 - Nazmi Zeybek, male_of_killer, Senem Zeybek, female_of_killed, Saniye Özdemir, female_of_killed, 02.09.2020

259 - 
Sinan T., male, S.K., female, S.K., female, 05.10.2020

261 - 
Serkan Titiz, Male, Figen Titiz, Female, 28.07.2020

262 - 
Unknown, Unknown, Mehmet Boz, Male, Unknown

263 - Necip Kocamanoğlu, male, Hüsne Kocamanoğlu, female, 16.30.2020

264 - 
John Doe, Male, Jane Doe, Female, 01.01.2020

265 - 
Ahmed Al-Hassan, Male, Asma Bouchane, Female, 28.03.2021

266 - 
Halil İbrahim E., male, Havva E., female, 05.07.2020

267 - 
John Doe, Male, Jane Doe, Female, 01.09.2020

268 - 
M.C., Male, Hulusi Gelişken, Male, 23.06.2020

269 - Sedat Dedeler, male, Elvan Dedeler, female, 15.07.2020

270 - 
Osman Kızılçelik, male, Fatma Kızılçelik, female, 24.04.2020

271 - 
name_of_killer: Unknown, gender_of_killer: Unknown, name_of_killed: Aynur Kanbur, gender_of_killed: Female, date: 24.04.2020

272 - 
Hasan Bat, Male, 30.05.2020

273 - 
Orhan Fırat, male, Abdurezzak Erol, male, 12.09.

381 - 
M.I., male, S.I., male, John Edward Donnelly, male, 03.06.2016

382 - 
Mehmet Irmak, Male, John Edward Donnelly, Male, 03.06.2020

384 - Mahmut K., male, Şevket Bülbül, male, 12.03.2016

385 - 
Emre Avşar, male, Suphiye Avşar, female, 15.07.2020

386 - 
Mustafa Gülcü, male, Haydar Meriç, male, İrfan Erbaşaran, male, 20.07.1990

387 - 
Sibel T., female, Engin T., male, 15.08.2020

388 - 
Name_of_killer: Ismi_Açıklanmayan_İşçi, Gender_of_killer: Unknown, Name_of_killed: Mehmet_Aytaç, Gender_of_killed

389 - 
Doğan Murat, male, Kenan Savacı, male, Ertuğrul Kandırmaz, male, 28.07.2020

390 - 
Nezir Yentar, Male, Azize Yentar, Female, 17.01.2014

391 - 
Engin Gülistan, male, Abdülbaki Aykut, male, 15.07.2020

392 - 
İbrahim Halil K., male, Döndü G., female, Kenan G., male, 15.08.2019.

393 - 
PKK, male, Menderes Özer, male, 15.07.2020

394 - 
C.D., male, Gülizar Turan, female, 14.08.2020

395 - 
Erkan B., male, Mete Yarar, male, Gökalp K., male, Gökhan K., male, 28.08.2020

396 - 
Ha

503 - 
H.Ö., male, İmdat O., male, 15.07.2020

504 - 
John Doe, Male, Jane Doe, Female, 01.30.2020

505 - 
Seyyit_Kılıç, Male, Yakup_Sağlam, Male, 15.07.2016

506 - 
G.E., male, Yılmaz Özçimen, male, 01.04.2020

507 - Barbaros Caddesi üzerinde meydana gelen olayda, Ethem D.’yi öldüren zanlıların, 24 yaşındaki İbrahim

508 - Unknown, Unknown, Unknown, Unknown, 15.07.2016

509 - 
Anonymous, Unknown, Mehmet Polent, Male, Unknown.

510 - 
Seyyit Kılıç, Male, Yakup Sağlam, Male, 15.07.2016.

511 - 
Akın S., male_killer, Tayfun S., male_killed, 15.03.2015

512 - 
Z.T., male, Nihat A., male, Adem Tunaoğlu, male, 15.07.2016

513 - 
unknown, unknown, Ertan Sancak, male, 15.07.2020

514 - 
R.A., male, A.U., female, O.U., male, 24.07.2020

515 - 
Thedarktower, Male, Unidentified, Male, Unknown

516 - 
Unknown, Unknown, Unknown, Unknown, Unknown

517 - 
Ogün Samast, male, 19.01.2007, M.D., male, Hrant Dink, male, 19.01.2007

518 - 
Anonymous, Male, Şükrü Çiçek, Male, Aşkın Cevizli, Male, 28.07.202

In [82]:
df["nlp analysis(name of killer, gender of killer, name of killed, gender of killed, date)"] = answers

In [91]:
df.to_excel(f"outputs/with nlp analysis/xlsx/scraped_urls_with_nlp_analysis-{file_name}.xlsx", engine='xlsxwriter')
df.to_csv(f"outputs/with nlp analysis/csv/scraped_urls_with_nlp_analysis-{file_name}.csv")